<a href="https://colab.research.google.com/github/MauricioCastroF/NLP-desafios/blob/main/Desaf%C3%ADo_2_NLP_CastroFMauricio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
from nltk.corpus import stopwords

# ***Dataset seleccionado***: IMDB dataset. Corresponde a comentarios/opiniones sobre el contenido de películas, recopilados por la plataforma IMDB

In [11]:
df = pd.read_csv('/content/sample_data/IMDB Dataset.csv', index_col = False, sep='/n')
df.head()

<ipython-input-11-512bcc6a53e1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/sample_data/IMDB Dataset.csv', index_col = False, sep='/n')
<ipython-input-11-512bcc6a53e1>:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv('/content/sample_data/IMDB Dataset.csv', index_col = False, sep='/n')


,"review,sentiment;;;"
0,One of the other reviewers has mentioned that ...
1,"""A wonderful little production. <br /><br />Th..."
2,"""I thought this was a wonderful way to spend t..."
3,Basically there's a family where a little boy ...
4,"""Petter Mattei's """"Love in the Time of Money""""..."


In [14]:
print("La cantidad de documentos es:", df.shape[0])

La cantidad de documentos es: 18498


1. ***Preprocesamiento***: Se procede a identificar/separar oraciones y palabras

In [15]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

2. Crear los vectores (word2vec)

In [16]:
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [18]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores 
                     negative=30,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [19]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [20]:
# Cantidad de filas/docs encontradas en el corpus
print("La cantidad de docs en el corpus es:", w2v_model.corpus_count)

La cantidad de docs en el corpus es: 18498


In [21]:
# Cantidad de words encontradas en el corpus
print("La cantidad de words distintas en el corpus es:", len(w2v_model.wv.index_to_key))

La cantidad de words distintas en el corpus es: 26238


3. ***Entrenar el modelo generador***

In [22]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=10,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 10663723.0
Loss after epoch 1: 8636533.0
Loss after epoch 2: 8387796.0
Loss after epoch 3: 8260168.0
Loss after epoch 4: 8298188.0
Loss after epoch 5: 8168264.0
Loss after epoch 6: 8068396.0
Loss after epoch 7: 7064156.0
Loss after epoch 8: 2498136.0
Loss after epoch 9: 2392112.0


(32022255, 43570440)

4. ***Ensayar***

Palabras que más se relacionan:

In [23]:
w2v_model.wv.most_similar(positive=["torture"], topn=20)

[('duckling', 0.6560611128807068),
 ('blink', 0.58382648229599),
 ('drink', 0.5803190469741821),
 ('bore', 0.5778264999389648),
 ('pretend', 0.5761638879776001),
 ('vomit', 0.5638636350631714),
 ('boil', 0.5569708943367004),
 ('eat', 0.5486410856246948),
 ('cause', 0.5430015921592712),
 ('defeat', 0.5359179973602295),
 ('scream', 0.5324810147285461),
 ('worry', 0.5321104526519775),
 ('carnage', 0.5292078852653503),
 ('nuke', 0.5255948305130005),
 ('satisfy', 0.5166351199150085),
 ('use', 0.5152300596237183),
 ('bother', 0.5135200023651123),
 ('smell', 0.5076311230659485),
 ('disgusting', 0.5050103664398193),
 ('hounds', 0.5049307346343994)]

In [25]:
w2v_model.wv.most_similar(positive=["summer"], topn=15)

[('weekend', 0.6868060827255249),
 ('reunion', 0.681554913520813),
 ('month', 0.6656711101531982),
 ('saturday', 0.6640940308570862),
 ('christmas', 0.6618317365646362),
 ('holiday', 0.6542794704437256),
 ('quarter', 0.6529728174209595),
 ('morning', 0.6522993445396423),
 ('night', 0.6485360264778137),
 ("year's", 0.6412177085876465),
 ('july', 0.637902557849884),
 ('afternoon', 0.6375389695167542),
 ('rainy', 0.6362606287002563),
 ('evening', 0.6337028741836548),
 ('august', 0.6291080117225647)]

Palabras que menos se relacionan:

In [26]:
w2v_model.wv.most_similar(negative=["wonderful"], topn=20)

[('somethings', 0.4510229825973511),
 ('000', 0.412354975938797),
 ('pound', 0.3892439007759094),
 ('dollars', 0.3879011869430542),
 ('yards', 0.37740665674209595),
 ('cell', 0.36913618445396423),
 ('enemy', 0.34772220253944397),
 ('housewives', 0.34687545895576477),
 ('min', 0.3422406315803528),
 ('ft', 0.33742275834083557),
 ('meters', 0.33488813042640686),
 ('suns', 0.33458203077316284),
 ('conveniently', 0.32877618074417114),
 ('press', 0.32811403274536133),
 ('cents', 0.32637375593185425),
 ('clerk', 0.32608330249786377),
 ('pounds', 0.32455572485923767),
 ('mins', 0.32211562991142273),
 ('commandments', 0.32109057903289795),
 ('foot', 0.31869444251060486)]

In [30]:
w2v_model.wv.most_similar(negative=["wrong"], topn=20)

[('mega', 0.4798754155635834),
 ("cinema's", 0.4601486325263977),
 ('martial', 0.45307594537734985),
 ('rapper', 0.4526119530200958),
 ('14th', 0.44385015964508057),
 ('philo', 0.42208030819892883),
 ('ivory', 0.42183980345726013),
 ('lone', 0.4209460914134979),
 ("disney's", 0.42048323154449463),
 ('dame', 0.40878960490226746),
 ('semi', 0.407375305891037),
 ('alexander', 0.40687236189842224),
 ('bafta', 0.40631353855133057),
 ('burt', 0.4030819535255432),
 ('garde', 0.4029640555381775),
 ("polanski's", 0.4023520052433014),
 ('marx', 0.40102022886276245),
 ("leone's", 0.4006943106651306),
 ('seth', 0.39790183305740356),
 ("trek's", 0.3977581858634949)]

5. Visualizar agrupación de vectores

In [28]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [29]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=50
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Conclusiones:
* 18 mil comentarios con 26 palabras diferentes, relacionadas con opiniones sobre películas fueron analizados mediante una técnica de Word Embedding
* Las palabras que se relacionaron más con "torture" fueron entre otras: parpadear, aburrido, vomit, etc. lo cual fue llamativo. 
* Las palabras que se relacionaron menos con "wrong" no fueron mayores a 0.497; 
* La palabra "Wonderful" tuvo relación negativa con palabras tales como: "dollar" y "enemy", lo cual fue llamativo
* En cuanto a la figura bidimencional de los embedddings se observó un agrupamiento de gran número de palabras en los cuadrantes de valores positivos de ambos ejes. 
